# Detection of PowerShell Execution via DLL
Detects PowerShell Strings applied to rundllas seen in PowerShdll.dll

## Rule Content
```
- title: Detection of PowerShell Execution via DLL
  id: 6812a10b-60ea-420c-832f-dfcc33b646ba
  status: experimental
  description: Detects PowerShell Strings applied to rundllas seen in PowerShdll.dll
  references:
  - https://github.com/p3nt4/PowerShdll/blob/master/README.md
  tags:
  - attack.execution
  - attack.t1086
  - car.2014-04-003
  author: Markus Neis
  date: 2018/08/25
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection1:
      Image:
      - '*\rundll32.exe'
    selection2:
      Description:
      - '*Windows-Hostprozess (Rundll32)*'
    selection3:
      CommandLine:
      - '*Default.GetString*'
      - '*FromBase64String*'
    condition: (selection1 or selection2) and selection3
  falsepositives:
  - Unknown
  level: high

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='((process_path.keyword:(*\\rundll32.exe) OR file_description.keyword:(*Windows\-Hostprozess\ \(Rundll32\)*)) AND process_command_line.keyword:(*Default.GetString* OR *FromBase64String*))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()